In [1]:
import pandas as pd
from dataset_fns import Dataset_Formatter, Dataset_Accessor, mp_Dataset_Formatter
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
import openeye as oe
from openeye import oechem
import numpy as n
from rdkit.Chem import Draw
from mordred import Calculator, descriptors
from pathlib import Path
import time
from multiprocessing import Pool
from rdkit.Chem import Descriptors


mk = Dataset_Formatter()
mp_mk = mp_Dataset_Formatter(n_processes=2)


In [2]:
data = mp_mk._load_data(mol_dir='/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/raw/',
                 filename='exp_test.inchi',
                 pymolgen=True,
                 prefix='HW-',
                 mol_type='smiles',
                 chunksize=10,
                 retain_ids=False)

In [9]:
data[0]

,Mol,Frag_SMILES,SMILES,Kekulised_SMILES,Lilly_rules_pass,Lilly_rules_warning,Lilly_rules_SMILES,Descriptors
ID,,,,,,,,
HW-1,<rdkit.Chem.rdchem.Mol object at 0x153d59b35f40>,None,Cc1cc(ccc1OC(=O)NC[NH+]2CCC(C2)C[NH2+]C)c3c(no...,CNCC1CCN(CNC(=O)OC2=CC=C(C3=C(C)ON=C3C)C=C2C)C1,False,(1 matches to 'phenolic_ester_or_carbamate'),CNCC1CCN(CNC(=O)OC2=CC=C(C3=C(C)ON=C3C)C=C2C)C1,"{'ABC': 20.78189539529507, 'ABCGG': 16.9436308..."
HW-2,<rdkit.Chem.rdchem.Mol object at 0x153d59b35ef0>,None,Cc1cc(ccc1NCS(=O)(=O)C2CCOCC2)c3c(noc3C)C,CC1=CC(C2=C(C)ON=C2C)=CC=C1NCS(=O)(=O)C1CCOCC1,True,D(50) aniline_h_newd,CC1=CC(=CC=C1NCS(=O)(=O)C1CCOCC1)C1=C(C)ON=C1C,"{'ABC': 19.514519721557935, 'ABCGG': 16.092772..."
HW-3,<rdkit.Chem.rdchem.Mol object at 0x153d59b26400>,None,CC=CC(=O)OC(C)(c1cc(cc(c1)Oc2cc(ccc2C)c3c(noc3...,CC=CC(=O)OC(C)(C1=CC(OC2=CC(C3=C(C)ON=C3C)=CC=...,False,(1 matches to 'michael_rejected'),CC=CC(=O)OC(C)(C1=CC(=CC(=C1)C(=O)NC)OC1=CC(=C...,"{'ABC': 27.46928521963825, 'ABCGG': 23.1440291..."
HW-4,<rdkit.Chem.rdchem.Mol object at 0x153d59b26ef0>,None,CCC[NH+]=C(NCC(c1ccccc1)O)Nc2cc(ccc2n3cccn3)c4...,CCCN=C(NCC(O)C1=CC=CC=C1)NC1=CC(C2=C(C)ON=C2C)...,True,D(60) too_many_atoms,CCCN=C(NCC(O)C1=CC=CC=C1)NC1=CC(=CC=C1N1C=CC=N...,"{'ABC': 26.248479616006502, 'ABCGG': 21.424419..."
HW-5,<rdkit.Chem.rdchem.Mol object at 0x153d5bc754a0>,None,CCCOc1ccc(cc1)C(c2cc(cc(c2)O)c3cc(ccc3C)c4c(no...,CCCOC1=CC=C(C(O)C2=CC(O)=CC(C3=CC(C4=C(C)ON=C4...,True,D(53) too_many_atoms,CCCOC1=CC=C(C(O)C2=CC(O)=CC(=C2)C2=CC(=CC=C2C)...,"{'ABC': 25.71971231978243, 'ABCGG': 19.8092006..."
HW-6,<rdkit.Chem.rdchem.Mol object at 0x153d5bdadb80>,None,Cc1cc(ccc1c2nnc3n2CC[NH+](C3)CNc4ncc5c(n4)nc[n...,CC1=CC(C2=C(C)ON=C2C)=CC=C1C1=NN=C2CN(CNC3=NC=...,False,(1 matches to 'ncn'),CC1=CC(=CC=C1C1=NN=C2CN(CNC3=NC=C4NC=NC4=N3)CC...,"{'ABC': 27.581812949339835, 'ABCGG': 19.965628..."
HW-7,<rdkit.Chem.rdchem.Mol object at 0x153d59b111d0>,None,Cc1ccc(cc1c2ccc(cn2)S(=O)(=O)NCCc3cncs3)c4c(no...,CC1=CC=C(C2=C(C)ON=C2C)C=C1C1=CC=C(S(=O)(=O)NC...,True,D(40) too_many_atoms,CC1=CC=C(C2=C(C)ON=C2C)C=C1C1=CC=C(S(=O)(=O)NC...,"{'ABC': 24.423827075343876, 'ABCGG': 19.177588..."
HW-8,<rdkit.Chem.rdchem.Mol object at 0x153d5bcb8b30>,None,Cc1ccc(cc1NCC2CCN(CC2)c3c4ccsc4ncn3)c5c(noc5C)C,CC1=CC=C(C2=C(C)ON=C2C)C=C1NCC1CCN(C2=NC=NC3=C...,True,D(90) aniline_h_newd:too_many_atoms,CC1=CC=C(C2=C(C)ON=C2C)C=C1NCC1CCN(C2=NC=NC3=C...,"{'ABC': 24.793825939113532, 'ABCGG': 18.632196..."
HW-9,<rdkit.Chem.rdchem.Mol object at 0x153d59affc70>,None,Cc1cc(ccc1c2ccc(cc2)c3cn(cc3C)C)c4c(noc4C)C,CC1=CC(C2=C(C)ON=C2C)=CC=C1C1=CC=C(C2=CN(C)C=C...,True,D(13) too_many_atoms,CC1=CC(=CC=C1C1=CC=C(C2=CN(C)C=C2C)C=C1)C1=C(C...,"{'ABC': 21.477071632663154, 'ABCGG': 16.994555..."


In [3]:
x, y = mp_mk._calculate_desc_df(descriptor_set='Mordred')

In [9]:
a, b = mp_mk._calculate_desc_df(descriptor_set='RDKit')

In [4]:
filt_df = mp_mk._filter_df()

/users/yhb18174/Recreating_DMTA/scripts/dataset/dataset_fns.py:1522: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_results = all_passing_mols[~df['SMILES'].isin(filtered_smi)]
/users/yhb18174/Recreating_DMTA/scripts/dataset/dataset_fns.py:1522: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_results = all_passing_mols[~df['SMILES'].isin(filtered_smi)]
/users/yhb18174/Recreating_DMTA/scripts/dataset/dataset_fns.py:1522: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_results = all_passing_mols[~df['SMILES'].isin(filtered_smi)]


In [7]:
full, desc = mp_mk._make_final_chunks(chunksize=10, gen_desc_chunks=True, descriptor_set='Mordred')
full[0]

Saving chunk 0 to:
NoneNone
Saving chunk 0 to:
NoneNone


,Frag_SMILES,SMILES,Kekulised_SMILES,Lilly_rules_pass,Lilly_rules_warning,Lilly_rules_SMILES,ABC,ABCGG,nAcid,nBase,...,MolWt,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,oe_logp,PFI
ID,,,,,,,,,,,,,,,,,,,,,
HW-2,None,Cc1cc(ccc1NCS(=O)(=O)C2CCOCC2)c3c(noc3C)C,CC1=CC(C2=C(C)ON=C2C)=CC=C1NCS(=O)(=O)C1CCOCC1,True,D(50) aniline_h_newd,CC1=CC(=CC=C1NCS(=O)(=O)C1CCOCC1)C1=C(C)ON=C1C,19.51452,16.092773,0.0,0.0,...,364.145678,7.431544,1633.0,39.0,132.0,155.0,8.840278,5.402778,2.342,4.342
HW-12,None,Cc1cc(ccc1NCS(=O)(=O)C2CCOCC2)c3c(noc3C)C,CC1=CC(C2=C(C)ON=C2C)=CC=C1NCS(=O)(=O)C1CCOCC1,True,D(50) aniline_h_newd,CC1=CC(=CC=C1NCS(=O)(=O)C1CCOCC1)C1=C(C)ON=C1C,19.51452,16.092773,0.0,0.0,...,364.145678,7.431544,1633.0,39.0,132.0,155.0,8.840278,5.402778,2.342,4.342
HW-22,None,Cc1cc(ccc1NCS(=O)(=O)C2CCOCC2)c3c(noc3C)C,CC1=CC(C2=C(C)ON=C2C)=CC=C1NCS(=O)(=O)C1CCOCC1,True,D(50) aniline_h_newd,CC1=CC(=CC=C1NCS(=O)(=O)C1CCOCC1)C1=C(C)ON=C1C,19.51452,16.092773,0.0,0.0,...,364.145678,7.431544,1633.0,39.0,132.0,155.0,8.840278,5.402778,2.342,4.342


In [43]:
calc = Calculator(descriptors, ignore_3D=True)
desc_names = [str(desc) for desc in calc.descriptors]


for names in desc_names:
    if 'naRing' in names:
        print(names)

naRing
